In [11]:
import pandas as pd

# Load data
orders = pd.read_csv('../data/orders.csv')
billing = pd.read_csv('../data/billing.csv')


In [12]:
from IPython.display import display

display(orders)
display(billing)

,ndc,drug_name,qty_ordered
0,1,Januvia,200
1,1,Januvia,300
2,2,Metformin,400
3,2,Metformin,200
4,3,Atorvastatin,500


,ndc,drug_name,insurance,bin_number,qty_billed
0,1,Januvia,Aetna,4336,250
1,2,Metformin,United,610011,600
2,1,Januvia,United,4336,300
3,3,Lisinopril,Cigna,3456,400


In [13]:
#aggregate order quantity 
orders_agg=(
    orders
     .groupby(['ndc', 'drug_name'])
    .agg(total_ordered=('qty_ordered', 'sum'))
    .reset_index()
)

In [14]:
orders.columns

Index(['ndc', 'drug_name', 'qty_ordered'], dtype='object')

In [15]:
#aggregate billing quantity 
billing_agg= (
    billing
    .groupby(['ndc', 'drug_name','bin_number', 'insurance'])
    .agg(total_billed=('qty_billed', 'sum'))
    .reset_index()
)

In [16]:
billing.columns

Index(['ndc', 'drug_name', 'insurance', 'bin_number', 'qty_billed'], dtype='object')

In [17]:
#join orders onto billing 
final_model=billing_agg.merge(
    orders_agg,
    on=['ndc','drug_name'],
    how='left'
    )
final_model['total_ordered']=final_model['total_ordered'].fillna(0)

In [18]:
#indicator logic 
def status_flag(row):
    if row['total_ordered']>=row["total_billed"]:
        return 'GREEN'
    elif row['total_ordered']>=0.9*row['total_billed']:
        return 'YELLOW'
    else:
        return 'RED'
final_model['status']=final_model.apply(status_flag,axis=1)

In [19]:
#useful columns 
final_model['coverage']=(
    final_model['total_ordered']/final_model['total_billed']
).round(2)

final_model['shortage_qty'] = (
    final_model['total_billed'] - final_model['total_ordered']
).clip(lower=0)


print(final_model)

   ndc   drug_name  bin_number insurance  total_billed  total_ordered status  \
0    1     Januvia        4336     Aetna           250          500.0  GREEN   
1    1     Januvia        4336    United           300          500.0  GREEN   
2    2   Metformin      610011    United           600          600.0  GREEN   
3    3  Lisinopril        3456     Cigna           400            0.0    RED   

   coverage  shortage_qty  
0      2.00           0.0  
1      1.67           0.0  
2      1.00           0.0  
3      0.00         400.0  
